# Dev contrastive knowledge assesment (CKA) notebook

* See [Capstone Repo](https://github.com/daniel-furman/Capstone) for more details
---

## Dependencies

In [3]:
!git clone https://github.com/daniel-furman/Capstone.git

Cloning into 'Capstone'...
remote: Enumerating objects: 110, done.
remote: Counting objects: 100% (110/110), done.
remote: Compressing objects: 100% (76/76), done.
remote: Total 110 (delta 26), reused 99 (delta 16), pack-reused 0
Receiving objects: 100% (110/110), 20.42 MiB | 7.38 MiB/s, done.
Resolving deltas: 100% (26/26), done.


In [4]:
!pip install -r /content/Capstone/src/cka/scripts/requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 56.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.3/76.3 MB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 104.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 28.1 MB/s eta 0:00:00


## Imports

In [5]:
import os
import numpy as np

import torch
from torch.nn.functional import softmax

from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    AutoModelForMaskedLM,
    T5Tokenizer,
    T5ForConditionalGeneration,
)

In [6]:
device = torch.device("cuda")

## Dev for new models

In [7]:
text = "The Milky Way is a [MASK] galaxy."

In [8]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModelForMaskedLM.from_pretrained("bert-base-uncased").to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [30]:
def probe_bert(model, tokenizer): #, target):
    """
    model: a pretrained bert model pulled in from HuggingFace (ie. bert-base-uncased)
    input_ids: the indices (in the vocabulary) of our left-context tokens
    target: the index (in the vocabulary) of the token we're gathering a prediction for

    return: a float indicating the likelihood of the target following the left-context
      according to the model in case of error, return None
    """

    # Call the tokenizer
    inputs = tokenizer(text, return_tensors="pt")
    mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]

    # Call the model
    logits = model(**inputs.to(device)).logits
    mask_token_logits = logits[0, mask_token_index, :]

    # Convert our prediction scores to a probability distribution with softmax
    probs = softmax(mask_token_logits, dim=-1)

    top_3_tokens = torch.topk(probs, 3, dim=1).indices[0].tolist()

    for token in top_3_tokens:
        print(text.replace(tokenizer.mask_token, tokenizer.decode([token])))
    print(probs.shape)

    probs = probs.detach().cpu().numpy()

    #return probs[target.item()]

In [31]:
probe_bert(model, tokenizer)

The Milky Way is a spiral galaxy.
The Milky Way is a dwarf galaxy.
The Milky Way is a satellite galaxy.
torch.Size([1, 30522])
